In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_baselines = pd.read_csv('../baslines.csv')[["embedder", "dataset", "metric_test"]]
df_distill = pd.read_csv('../ckpt.csv')[["epoch", "dataset", "metric_test"]]
last_epoch = df_distill.groupby("dataset").epoch.max()
df_distill = df_distill[df_distill.epoch.isin(last_epoch)]
df_distill = df_distill.rename(columns={"epoch": "embedder"})

In [ ]:
df_or = pd.concat([df_baselines, df_distill]).groupby(["dataset", "embedder"]).metric_test.mean().reset_index()
df = df_or.pivot_table(index="embedder", columns="dataset", values="metric_test")
df.dropna(axis=1, inplace=True)
# drop column and index names
df.index.name = None

df

In [ ]:
models = df.mean(axis=1).sort_values(ascending=False)[:15].index.tolist()

In [ ]:
# Bold the best value for each dataset
def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

df.transpose()[models].style.apply(highlight_max, axis=1)



In [ ]:
df_or
# Remove all dataset with missing values
df_or = df_or[df_or.dataset.isin(df.columns)]

In [ ]:
fig = plt.figure(figsize=(5, 15))
N_MODELS = 10
df_tmp = df_or[
    df_or.embedder.isin(models[:N_MODELS])
]
df_tmp.metric_test = df_tmp.metric_test - df_or.groupby("dataset").metric_test.transform("min")

sns.barplot(data=df_tmp, y="dataset", x="metric_test", hue="embedder", hue_order=models[:N_MODELS])